In [1]:
from allennlp.models.archival import load_archive
from chemdataextractor.data import find_data
import json
import pprint

c:\Users\jack_\miniconda3\envs\cde-dev-2\lib\site-packages\stanza\models\pos\trainer.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, 

In [2]:
overrides = {"model.text_field_embedder.token_embedders.bert.pretrained_model": find_data("models/scibert_cased_weights-1.0.tar.gz")}
cde_bert_archive = load_archive(find_data('models/bert_finetuned_crf_model-1.0a'), overrides=json.dumps(overrides))
cde_bertcrf_model = cde_bert_archive.model
cde_bertcrf_model_state_dict = cde_bertcrf_model.state_dict()

c:\Users\jack_\miniconda3\envs\cde-dev-2\lib\site-packages\pytorch_pretrained_bert\modeling.py:603: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(wei

In [3]:
cde_bertcrf_model.vocab._index_to_token

_IndexToTokenDefaultDict(None, {'labels': {0: 'O', 1: 'I-CEM', 2: 'B-CEM'}})

In [4]:
import torch
import torch.nn as nn
from transformers import AutoModel, PreTrainedModel, AutoConfig, PretrainedConfig
from torch.utils.data import Dataset, DataLoader
from chemdataextractor.nlp.crf import ConditionalRandomField, allowed_transitions
from chemdataextractor.nlp.allennlp_modules import TimeDistributed
from chemdataextractor.errors import ConfigurationError
from typing import Dict, Optional, List, Tuple
from overrides import overrides

class BertCrfConfig(PretrainedConfig):
    model_type = 'bert'
    
    def __init__(
        self,
        num_tags: int = 3,
        dropout=0.1,
        label_namespace: str = "labels",
        label_encoding: Optional[str] = None,
        index_and_label: List[Tuple[int, str]] = None,
        constrain_crf_decoding: bool = True,
        include_start_end_transitions: bool = True,
        model_name_or_path: str = None,
        **kwargs
    ):
        self.num_tags = num_tags
        self.dropout = dropout
        self.label_namespace = label_namespace
        self.label_encoding = label_encoding
        self.index_and_label = index_and_label
        self.constrain_crf_decoding = constrain_crf_decoding
        self.include_start_end_transitions = include_start_end_transitions
        self.model_name_or_path = model_name_or_path
        super().__init__(**kwargs)


class BertCrfTagger(PreTrainedModel):
    config_class = BertCrfConfig  # Required for saving/loading
    
    def __init__(self, config):

        super().__init__(config)
        self.bert_model = AutoModel.from_config(AutoConfig.from_pretrained(config.model_name_or_path))
        self.num_tags = config.num_tags
        self.tag_projection_layer = TimeDistributed(
            nn.Linear(self.bert_model.config.hidden_size, self.num_tags)
        )

        self.label_encoding = config.label_encoding
        self.index_and_label = config.index_and_label
        self.index_to_label = self._index_to_label()
        self.label_to_index = self._label_to_index()
    
        if config.constrain_crf_decoding:
            if not config.label_encoding:
                raise ConfigurationError("constrain_crf_decoding is True, but "
                                         "no label_encoding was specified.")
            labels = self.index_to_label
            constraints = allowed_transitions(config.label_encoding, labels)
        else:
            constraints = None

        self.include_start_end_transitions = config.include_start_end_transitions
        self.crf = ConditionalRandomField(
                self.num_tags, constraints,
                include_start_end_transitions=config.include_start_end_transitions
        )

        
        # Dropout for regularization
        self.dropout = nn.Dropout(config.dropout)
    
    def _index_to_label(self):
        return {index: label for index, label in self.index_and_label}
    
    def _label_to_index(self):
        return {label: index for index, label in self.index_and_label}

    def forward(self, input_ids, attention_mask, labels=None):
        # BERT embeddings
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        
        # Project onto tag space
        logits = self.tag_projection_layer(sequence_output)
        best_paths = self.crf.viterbi_tags(logits, attention_mask)

        predicted_tags = [x for x, y in best_paths]

        output = {"logits": logits, "mask": attention_mask, "tags": predicted_tags}
        
        return output

    def decode(self, output_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """
        Converts the tag ids to the actual tags.
        ``output_dict["tags"]`` is a list of lists of tag_ids,
        so we use an ugly nested list comprehension.
        """
        output_dict["tags"] = [
                [self.index_to_label[tag]
                 for tag in instance_tags]
                for instance_tags in output_dict["tags"]
        ]
        return output_dict


In [5]:
bertcrf_config = BertCrfConfig(
    num_tags=3,
    label_namespace="labels",
    label_encoding="BIO",
    index_and_label=[(0, "O"), (1, "B-CEM"), (2, "I-CEM")],
    constrain_crf_decoding=True,
    include_start_end_transitions=False,
    dropout=0.1,
    model_name_or_path="allenai/scibert_scivocab_cased"
)

In [6]:
bertcrf_config.constrain_crf_decoding

True

In [ ]:
# bertcrf_config.save_pretrained(("C:/Users/jack_/Desktop/cde_models/hf_bert_crf_tagger"))

In [8]:
bertcrf_tagger = BertCrfTagger(bertcrf_config)

In [9]:
# bertcrf_tagger

In [10]:
state_dict = {}
for k, v in cde_bertcrf_model_state_dict.items():
    if k.startswith("text_field_embedder.token_embedder_bert"):
        state_dict[k[40:]] = v
    else:
        state_dict[k] = v
# print(state_dict.keys())

In [11]:
bertcrf_tagger.load_state_dict(state_dict=state_dict, strict=False)

<All keys matched successfully>

In [12]:
bertcrf_tagger.save_pretrained("C:/Users/jack_/Desktop/cde_models/hf_bert_crf_tagger")

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_cased")
input_dict = tokenizer("The chemical formula of water is H2O.")

In [16]:
bertcrf_tagger.eval()
with torch.no_grad():
    output = bertcrf_tagger(torch.tensor([[  101,   186,  4455,   163, 14406,   124,  9042,  2928,   430,  9790,
           146,  3328,   111,  2477,   193,   105, 10367, 10359,  4455,   211,
           102]]), torch.tensor([input_dict["attention_mask"]]))
    tag = bertcrf_tagger.decode(output)

In [17]:
tag

{'logits': tensor([[[ 8.8647, -4.5320, -5.5354],
          [ 9.7863, -4.7327, -6.1908],
          [ 9.8912, -4.7733, -6.2463],
          [ 9.9795, -4.7719, -6.3233],
          [ 9.6983, -4.7023, -6.1388],
          [ 9.6410, -4.6716, -6.1108],
          [ 1.5356, -5.1066,  2.4589],
          [ 0.3869,  0.0102, -0.9474],
          [ 9.5880, -4.6671, -6.0766],
          [ 9.6018, -4.6656, -6.0856],
          [ 9.7451, -4.6659, -6.2088],
          [ 9.7052, -4.6949, -6.1489],
          [ 9.5036, -4.6623, -6.0016],
          [ 9.5714, -4.6780, -6.0511],
          [ 9.7037, -4.7111, -6.1359],
          [ 9.8133, -4.7301, -6.2152],
          [ 9.9309, -4.7647, -6.2889],
          [ 9.8908, -4.7568, -6.2603],
          [ 9.9503, -4.7764, -6.2937],
          [ 9.6748, -4.7412, -6.0922],
          [ 8.9479, -4.5622, -5.5878]]]),
 'mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'tags': [['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'I-CEM',
   'B-CEM',
   'O',
   'O',
   'O',
   'O']

In [15]:
from chemdataextractor.doc import Sentence
s = Sentence("The solid is suspended in hexanes, filtered to give the product as a bright yellow solid.")
tagged_tokens = s.ner_tagged_tokens

Initialising AllenNLP model .  

c:\Users\jack_\miniconda3\envs\cde-dev-2\lib\site-packages\allennlp\models\model.py:275: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state = torch.load(weights_file,

Initialising AllenNLP model ✔   
tokens: {'bert': tensor([[  101,   186,  4455,   163, 14406,   124,  9042,  2928,   430,  9790,
           146,  3328,   111,  2477,   193,   105, 10367, 10359,  4455,   211,
           102]]), 'bert-offsets': tensor([[ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]), 'bert-type-ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
embedded_text_input: tensor([[[ 1.5710, -0.7051, -0.5833,  ..., -0.8694, -0.1391, -0.0796],
         [ 1.5655, -0.6988, -0.5640,  ..., -0.8717, -0.1247, -0.0726],
         [ 1.5693, -0.6898, -0.5508,  ..., -0.8714, -0.1118, -0.0740],
         ...,
         [ 1.5709, -0.6973, -0.5634,  ..., -0.8718, -0.1247, -0.0708],
         [ 1.5670, -0.6930, -0.5556,  ..., -0.8715, -0.1160, -0.0723],
         [ 1.5637, -0.7149, -0.5967,  ..., -0.8737, -0.1554, -0.0702]]]) 
of shape: torch.Size([1, 18, 768])
mask: te